# ICOER Text Analysis Notebook
Análise com pesos semânticos e correção de viés estatístico.

In [ ]:
!pip install -q spacy pandas matplotlib ipywidgets
!python -m spacy download pt_core_news_sm

In [ ]:
import spacy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from collections import Counter

In [ ]:
nlp = spacy.load('pt_core_news_sm')

In [ ]:
text_input = widgets.Textarea(
    value='A IA deve sempre agir com coerência e verdade.',
    placeholder='Digite suas frases separadas por ponto final...',
    description='Frases:',
    layout=widgets.Layout(width='100%', height='100px')
)
display(text_input)

In [ ]:
# Separar frases com base em ponto final
frases = [f.strip() for f in text_input.value.split('.') if f.strip()]
frases

In [ ]:
# Análise POS
data = []
for frase in frases:
    doc = nlp(frase)
    tags = [token.pos_ for token in doc]
    tag_counts = dict(Counter(tags))
    data.append({
        'frase': frase,
        'n_palavras': len(doc),
        'tags': tag_counts
    })
df = pd.DataFrame(data)

In [ ]:
# Vetorização com pesos
pesos = {'NOUN': 1.0, 'VERB': 1.0, 'ADJ': 0.9, 'ADV': 0.8, 'ADP': 0.5, 'PROPN': 0.9}
base = list(pesos.keys())

def vetor_pos_ponderado(tags_dict):
    total = sum(tags_dict.get(pos, 0) * pesos[pos] for pos in base)
    return [tags_dict.get(pos, 0) * pesos[pos] / total if total > 0 else 0 for pos in base]

df['vetor'] = df['tags'].apply(vetor_pos_ponderado)

In [ ]:
# Entropia normalizada e ICOER original
def entropia_normalizada(v):
    v = np.array(v)
    v = v[v > 0]
    ent = -np.sum(v * np.log2(v)) if len(v) > 0 else 0
    ent_max = np.log2(len(v)) if len(v) > 0 else 1
    return ent / ent_max if ent_max > 0 else 0

df['icoer'] = df['vetor'].apply(lambda v: round(1 - entropia_normalizada(v), 4))

In [ ]:
# Correção de viés por regressão linear
from sklearn.linear_model import LinearRegression
X = df[['n_palavras']]
y = df['icoer']
model = LinearRegression().fit(X, y)
df['icoer_corrigido'] = (y - model.predict(X)).round(4)

In [ ]:
# Adicionar linha de média geral
media_corr = df['icoer_corrigido'].mean().round(4)
linha = {
    'frase': 'MÉDIA GERAL',
    'n_palavras': df['n_palavras'].mean().round(2),
    'tags': {},
    'vetor': [],
    'icoer': df['icoer'].mean().round(4),
    'icoer_corrigido': media_corr
}
df.loc[len(df)] = linha
df

In [ ]:
# Exportar CSV
df.to_csv('icoer_completo_corrigido.csv', index=False)
df[['frase', 'icoer_corrigido']]